Import library

In [ ]:
!pip install keras_tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
# arrays
import numpy as np

# fits
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import simple_norm
from astropy.table import Table

# plotting
from matplotlib import pyplot as plt

# keras
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, LSTM, Input, GRU
from tensorflow.keras.layers import LeakyReLU
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from numpy import unique
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import datetime
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from keras import backend as K

# sklearn (for machine learning)
from sklearn.model_selection import train_test_split
from sklearn import metrics

#Drive
from google.colab import drive


In [ ]:
from keras.layers import  BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

#Some usefull function

In [ ]:
def plotta_gaussiana_errore(pred1, pred2, pred3, label1, label2, label3):
    # Compute errors between prediction and label
    error1 = pred1 - label1
    error2 = pred2 - label2
    error3 = pred3 - label3

    # Calculate the mean and standard deviation of the errors
    mean1 = np.mean(error1)
    std1 = np.std(error1)
    mean2 = np.mean(error2)
    std2 = np.std(error2)
    mean3 = np.mean(error3)
    std3 = np.std(error3)
    # Generate a range of x values for the plot
    x = np.arange(-5000, 5000, 500)

    # Calculate the Gaussian curves of the errors
    y1 = (1 / (std1 * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean1) / std1) ** 2)
    y2 = (1 / (std2 * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean2) / std2) ** 2)
    y3 = (1 / (std3 * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean3) / std3) ** 2)

    # Plot the Gaussian curves of the errors
    plt.figure(figsize=(16, 10))
    plt.plot(x, y1, label='All Stars', color='green')
    plt.plot(x, y2, label='Blue Stars', color='blue')
    plt.plot(x, y3, label='Red Stars', color='red')
    plt.axvline(x=mean1, color='green', linestyle='--', label='Mean Error all stars')
    plt.axvline(x=mean2, color='blue', linestyle='--', label='Mean Error Blue stars')
    plt.axvline(x=mean3, color='red', linestyle='--', label='Mean Error Red stars')
    plt.xlabel('Error')
    plt.ylabel('Probability distribution')
    plt.title('Error distribution')
    plt.legend()
    plt.xticks(x)
    plt.show()


In [ ]:
def calcola_errore(pred, gt):
    # Calculate the error as the difference between predictions and ground truth
    error = pred - gt
    # Calculate the mean error
    mean_error = np.mean(error)
    # Calculate the variance of the error
    error_variance = np.var(error)
    return mean_error, error_variance

In [ ]:
def filter_and_plot(predictions, labels, y_bassa=0, y_alta=1):
    x= np.linspace(y_bassa, y_alta, 1000)
    plt.figure(figsize=(16, 16))
    plt.grid()
    plt.plot(x, x, label='Linea di riferimento')
    plt.plot(labels, predictions, 'x', label='Valori predetti')
    plt.xlabel('Fe/H reale')
    plt.ylabel('Fe/H predetto')
    plt.title('Confronto tra valori predetti e reali per Fe/H')
    plt.ylim(y_bassa, y_alta)
    plt.legend()
    plt.show()

Load the observed stellar spectra, each of them is labelled with a temperature and Fe/H
The CNN will be trained to predict the stellar temperature values.

Mount drive

In [ ]:
drive.mount('/content/drive/', force_remount=True)
path_data = "/content/drive/MyDrive/Colab Notebooks/environment/Galaxi marging/Telescope Data/Spectroscopy CNN/"

Mounted at /content/drive/


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/environment/22-03-23EX/Telescope Data/Spectroscopy CNN"

ls: cannot access '/content/drive/MyDrive/Colab Notebooks/environment/22-03-23EX/Telescope Data/Spectroscopy CNN': No such file or directory


#read the files

In [ ]:
file_name = 'XSL_labels.fits'
hdu = fits.open(path_data+file_name)
Labels = Table(hdu[1].data)

The spectra are sampled in 1000 pixels from 350 to 950 nm, and are normalized to 1 at 500 nm

In [ ]:
#Read the spectra
hdu = fits.open(path_data+'XSHOOTER_spec_lib.fits')
spectra = hdu[0].data
wave = np.linspace(350,950,1000)

In [ ]:
# set the random seed to get the same random set of images each time, or comment it out to get different ones!
np.random.seed(101)

# select 16 random image indices:
example_ids = np.random.choice(len(spectra), 4)
print(Labels['Teff'][example_ids])

# initialize your figure
fig, ax= plt.subplots(nrows=2, ncols=1, figsize=(14, 5))

# loop through the randomly selected images and plot with labels
for i in range(2):
    ax[i].plot(wave,spectra[example_ids[2*i+0],:,0])
    ax[i].plot(wave,spectra[example_ids[2*i+1],:,0])

plt.show()

We will denote the input spectra as X and their corresponding labels (temperature) as y, following the convention used by sklearn.



In [ ]:
# Save the spectra and labels for the external temperature and iron-hydrogen ratio
X = spectra
Y = np.array(Labels['Teff'])
Y_FH= np.array(Labels['__Fe_H_'])

# Split the red stars from the blue stars and their respective labels
stelle_rosse = X[np.where(Y < 4000)[0]]
stelle_azzurre = X[np.where(Y >4000)[0]]

#temperature
Teff_rosse = Y[np.where(Y < 4000)[0]]
Teff_azzurre = Y[np.where(Y > 4000)[0]]

#iron-hydrogen ratio
FH_rosse = Y_FH[np.where(Y < 4000)[0]]
FH_azzurre = Y_FH[np.where(Y > 4000)[0]]

In [ ]:
plt.figure(figsize=(10, 1))
#plot histogram of Teff
plt.hist([Y], bins=1000, label=['Teff'], color='red')
plt.legend()
plt.xlabel('Valore')
plt.ylabel('Frequenza')
plt.ylim(0, 20)
plt.show()

In [ ]:
plt.figure(figsize=(10, 1))
# Plot histogram of Fe/H
plt.hist([Y_FH], bins=1000, label=['Fe/H'], color='red')
plt.legend()
plt.xlabel('Valore')
plt.ylabel('Frequenza')
plt.ylim(0, 20)
plt.show()

In [ ]:
#Nomalize Teff between 0 and 1
norm = np.max(Labels['Teff'])
Y_norm = Y/norm
Teff_rosse = Teff_rosse/ norm
Teff_azzurre = Teff_azzurre / norm

In [ ]:
#find the nan values and remuve them
bad = np.any(np.isnan(X[:,:,0]), axis=1)
bad_rosse = np.any(np.isnan(stelle_rosse[:,:,0]), axis=1)
bad_azzurre = np.any(np.isnan(stelle_azzurre[:,:,0]), axis=1)
#delete variables unsuefull
X = X[~bad,...]
Y_norm = Y_norm[~bad,...]
Y_FH = Y_FH[~bad, ...]
stelle_rosse = stelle_rosse[~bad_rosse,...]
stelle_azzurre = stelle_azzurre[~bad_azzurre,...]
FH_rosse = FH_rosse[~bad_rosse,...]
FH_azzurre = FH_azzurre[~bad_azzurre,...]
Teff_rosse = Teff_rosse[~bad_rosse,...]
Teff_azzurre = Teff_azzurre[~bad_azzurre,...]

In [ ]:
#split for teff
training_data_X, testing_data_X, training_data_Y, testing_data_Y = train_test_split(X, Y_norm, test_size=0.2, random_state=42)
#Split for FH
train_FH, test_FH, train_y_FH, test_y_FH = train_test_split(X, Y_FH, test_size=0.2, random_state=42)
#split for FH rosse e azzurre
train_rosse_FH, test_rosse_FH, y_train_rosse_FH, y_test_rosse_FH = train_test_split(stelle_rosse, FH_rosse, test_size=0.2, random_state=42)
train_azzurre_FH, test_azzurre_FH, y_train_azzurre_FH, y_test_azzurre_FH = train_test_split(stelle_azzurre, FH_azzurre, test_size=0.2, random_state=42)
series_size = len(training_data_X[0,:,0])

In [ ]:
#Normalize Fe/H values between 0 and 1
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler_FH = MinMaxScaler()
train_y_FH = train_y_FH.reshape(-1, 1)
test_y_FH = test_y_FH.reshape(-1,1)
y_test_rosse_FH = y_test_rosse_FH.reshape(-1,1)
y_test_azzurre_FH = y_test_azzurre_FH.reshape(-1,1)
scaler_FH.fit(train_y_FH)

train_y_FH = scaler_FH.transform(train_y_FH)
test_y_FH = scaler_FH.transform(test_y_FH)

y_test_rosse_FH = scaler_FH.transform(y_test_rosse_FH)
y_test_azzurre_FH = scaler_FH.transform(y_test_azzurre_FH)



#Prediction of Teff

define model with only convolutional layer

In [ ]:
def define_model(initial_input_shape):
  model = Sequential()
  model.add(Conv1D(3, 15, activation='relu',  padding='same', input_shape=initial_input_shape))
  model.add(MaxPooling1D(2))
  model.add(Conv1D(3, 13, activation='relu',  padding='same' ) )
  model.add(Flatten(name='flatten_layer'))
  layer = model.get_layer('flatten_layer')
  model.add(Dense(layer.output_shape[1], activation='relu'))
  model.add(Dense(1, activation='linear'))
  opt = Adam(learning_rate=0.0001)
  model.compile(optimizer=opt, loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae']) #now compile the model with all the ingredients
  model.summary()
  return model

input_shape = np.shape(training_data_X[1])
CNN = define_model(input_shape)

define model with lstm layer

In [ ]:
def define_model(initial_input_shape):
    model = Sequential()
    model.add(Conv1D(3, 15, activation='relu', padding='same', input_shape=initial_input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(3, 13, activation='relu', padding='same'))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
    model.summary()
    return model

input_shape = np.shape(training_data_X[1])
CNN = define_model(input_shape)

define model with Gru

In [ ]:
def define_model(initial_input_shape):
    model = Sequential()
    model.add(Conv1D(3, 15, activation='relu', padding='same', input_shape=initial_input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(3, 13, activation='relu', padding='same'))
    model.add(GRU(32, activation='relu', return_sequences=True))
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=opt, loss=tf.keras.losses.MeanAbsoluteError(), metrics=['mae'])
    model.summary()

    return model

input_shape = np.shape(training_data_X[1])
CNN = define_model(input_shape)

model with only fully connetted layer

In [ ]:
def define_model(input_shape):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mae', metrics=['mae'])
    model.summary()
    return model

input_shape = np.shape(training_data_X[1])
CNN = define_model(input_shape)

Trasformer

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.activations import gelu

def scaled_dot_product_attention(q, k, v, mask=None):
    # Calculate the dot product between query and key matrices
    matmul_qk = K.batch_dot(q, k, axes=[2, 2])
    # Get the dimension of the key matrix
    dk = K.cast(K.shape(k)[-1], dtype=K.floatx())
    # Scale the attention logits by the square root of dk
    scaled_attention_logits = matmul_qk / K.sqrt(dk)
    # Apply mask to attention logits if provided
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    # Apply softmax to obtain attention weights
    attention_weights = K.softmax(scaled_attention_logits, axis=-1)
    # Calculate the weighted sum of the value matrix
    output = K.batch_dot(attention_weights, v)
    return output, attention_weights

def transformer_encoder(inputs, d_model, num_heads, ff_dim, dropout):
    # Multi-head self-attention mechanism
    attention = inputs
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)

    # Apply scaled dot-product attention
    attention, _ = scaled_dot_product_attention(attention, attention, attention)

    # Residual connection and layer normalization
    outputs = LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feed-forward neural network
    ffn = outputs
    ffn = Dense(ff_dim, activation=gelu)(ffn)
    ffn = Dropout(dropout)(ffn)
    ffn = Dense(d_model)(ffn)
    ffn = Dropout(dropout)(ffn)

    # Residual connection and layer normalization
    outputs = LayerNormalization(epsilon=1e-6)(outputs + ffn)

    return outputs

def define_model(input_shape, d_model, num_heads, ff_dim, num_encoders, dropout):
    # Define the input layer
    inputs = Input(shape=input_shape)

    # Apply transformer encoder blocks
    x = inputs
    for _ in range(num_encoders):
        x = transformer_encoder(x, d_model, num_heads, ff_dim, dropout)

    # Extract the final output from the last encoder block
    outputs = Dense(1, activation='linear')(x[:, -1, :])

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    # Compile the model with optimizer, loss function, and metrics
    model.compile(optimizer = opt, loss='mae', metrics=['mae'])

    # Print model summary
    model.summary()

    return model


input_shape = np.shape(training_data_X[1]) # Shape of the spectra
d_model = 64  # Dimension of the model
num_heads = 10  # Number of heads of attention
ff_dim = 128  # Dimension of feed-forward layer
num_encoders = 2  # Number of encoders in the Transformer
dropout = 0.1  # Dropout value

CNN = define_model(input_shape, d_model, num_heads, ff_dim, num_encoders, dropout)


In [ ]:
n_epochs = 10
batch_size = 8
#fit the model
history = CNN.fit(training_data_X, training_data_Y,
                  batch_size=batch_size,
                  validation_split=0.3,
                  epochs=n_epochs, verbose=1)


In [ ]:
#Plot loss and validation loss
plt.figure(figsize=(8, 8))
plt.title('Mean Average Error')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='validation')
plt.xlabel('Epoch')
plt.legend()
plt.show()
plt.clf()



In [ ]:
#make prediction
y_pred= CNN.predict(testing_data_X, verbose = True, batch_size=64)

3/3 [==============================] - 4s 1s/step


In [ ]:
#plot prediction
plt.figure(figsize=(10, 6))
test_norm=testing_data_Y*norm
pred_norm = y_pred*norm
x=np.linspace(0,30000,1000)
plt.grid()
plt.plot(x,x)
plt.plot(test_norm, pred_norm , 'x')

In [ ]:
#plot prediction whithout outliers
filter_and_plot(pred_norm, test_norm, 0 , 30000)

In [ ]:
#compute MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_norm, pred_norm)
print("Mean Absolute Error (MAE):", mae)

In [ ]:
#compute mean error and variance
errore , varianza = calcola_errore(pred_norm, test_norm)
errore, varianza

(712.6698019350536, 16912017.001289435)

##Predict only Red stars

In [ ]:
#make prediction for red star
y_pred_rosse = CNN.predict(stelle_rosse, verbose = True, batch_size=64)
#denormalization outputs and labels
y_pred_rosse =  y_pred_rosse*norm
Teff_rosse = Teff_rosse * norm

4/4 [==============================] - 2s 485ms/step


In [ ]:
#compute mean error and variance
errore , varianza = calcola_errore(y_pred_rosse, Teff_rosse)
errore, varianza

##Predic only not red stars

In [ ]:
#compute outputs
y_pred_azzurre = CNN.predict(stelle_azzurre, verbose = True, batch_size=64)
#denormalization outputs and labels
y_pred_azzurre = y_pred_azzurre* norm
Teff_azzurre = Teff_azzurre*norm

9/9 [==============================] - 3s 387ms/step


In [ ]:
#compute mean error and variance
errore , varianza = calcola_errore(y_pred_azzurre, Teff_azzurre)
errore, varianza

In [ ]:
#plot error
plotta_gaussiana_errore(pred_norm,  y_pred_azzurre, y_pred_rosse, test_norm, Teff_azzurre, Teff_rosse)

##Hyperparameter Tuning for convolutional layer for Teff

define model for the hyperparameter tuning

In [ ]:
def def_model(hp):

  model = Sequential()
  hp_Num_Kernel=hp.Int('kernel', min_value=3, max_value=21, step=2)
  hp_kernel_size=hp.Int('kernel_size',min_value=3, max_value=17, step=2)
  model.add(Conv1D(hp_Num_Kernel,hp_kernel_size, activation='relu',  padding='same', input_shape=input_shape))

  hp_Num_MaxPool=hp.Int('maxpoolSize', min_value=2, max_value=4, step=2)
  model.add(MaxPooling1D(2))

  hp_Num_Kernel1=hp.Int('kernel1', min_value=3, max_value=21, step=2)
  hp_kernel_size1=hp.Int('kernel_size1',min_value=3, max_value=17, step=2)
  model.add(Conv1D(hp_Num_Kernel1,hp_kernel_size1, activation='relu',  padding='same'))

  model.add(Flatten(name='flatten_layer')) #now we flat the maxpooling output to match the fully connected Dense layer input
  layer = model.get_layer('flatten_layer')

  model.add(Dense(layer.output_shape[1], activation='relu'))

  #scelta la sigmoide perche classificazione solo si o no e non con molte classi
  model.add(Dense(1, activation='sigmoid'))

  hp_learningRate=hp.Float('learning rate', min_value=0.001, max_value=0.01, step=0.02 )
  opt = SGD(hp_learningRate)

  model.compile(optimizer=opt, loss=tf.keras.losses.MeanSquaredError(), metrics=['mse']) #now compile the model with all the ingredients
  model.summary()

  return model

In [ ]:
tuner = kt.Hyperband(def_model, objective='val_loss', max_epochs=5, factor=3)

In [ ]:
tuner.search_space_summary()

In [ ]:
stop_early= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
n_epochs = 100
#search of best parameters
tuner.search(training_data_X,training_data_Y,batch_size=batch_size, epochs=n_epochs, validation_split=0.2, callbacks=[stop_early] )
best_hp=tuner.get_best_hyperparameters()[0]
hmodel=tuner.hypermodel.build(best_hp)
hmodel.summary()
#values of best model
best_hp.values

In [ ]:
#fit model
history=hmodel.fit(training_data_X,training_data_Y,
                  batch_size=batch_size,
                  validation_split=0.3,
                  epochs=n_epochs, callbacks=stop_early, verbose=1)

In [ ]:
#make outputs
pred= hmodel.predict(testing_data_X)

5/5 [==============================] - 0s 16ms/step


In [ ]:
#plot prediction
plt.figure(figsize=(10, 6))
test_norm = testing_data_Y*norm
pred_norm = y_pred*norm
x = np.linspace(0,30000,1000)
plt.grid()
plt.plot(x,x)
plt.plot(test_norm, pred_norm , 'x')

In [ ]:
#plot prediction whithout outliers
filter_and_plot(pred_norm, test_norm, 0 , 30000)

#Prediction of Fe H

Trasformer

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.activations import gelu

def scaled_dot_product_attention(q, k, v, mask=None):
    # Calculate the dot product between query and key matrices
    matmul_qk = K.batch_dot(q, k, axes=[2, 2])
    # Get the dimension of the key matrix
    dk = K.cast(K.shape(k)[-1], dtype=K.floatx())
    # Scale the attention logits by the square root of dk
    scaled_attention_logits = matmul_qk / K.sqrt(dk)
    # Apply mask to attention logits if provided
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    # Apply softmax to obtain attention weights
    attention_weights = K.softmax(scaled_attention_logits, axis=-1)
    # Calculate the weighted sum of the value matrix
    output = K.batch_dot(attention_weights, v)
    return output, attention_weights

def transformer_encoder(inputs, d_model, num_heads, ff_dim, dropout):
    # Multi-head self-attention mechanism
    attention = inputs
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)
    attention = Dense(d_model, activation=gelu)(attention)
    attention = Dropout(dropout)(attention)

    # Apply scaled dot-product attention
    attention, _ = scaled_dot_product_attention(attention, attention, attention)

    # Residual connection and layer normalization
    outputs = LayerNormalization(epsilon=1e-6)(inputs + attention)

    # Feed-forward neural network
    ffn = outputs
    ffn = Dense(ff_dim, activation=gelu)(ffn)
    ffn = Dropout(dropout)(ffn)
    ffn = Dense(d_model)(ffn)
    ffn = Dropout(dropout)(ffn)

    # Residual connection and layer normalization
    outputs = LayerNormalization(epsilon=1e-6)(outputs + ffn)

    return outputs

def define_model(input_shape, d_model, num_heads, ff_dim, num_encoders, dropout):
    # Define the input layer
    inputs = Input(shape=input_shape)

    # Apply transformer encoder blocks
    x = inputs
    for _ in range(num_encoders):
        x = transformer_encoder(x, d_model, num_heads, ff_dim, dropout)

    # Extract the final output from the last encoder block
    outputs = Dense(1, activation='linear')(x[:, -1, :])

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    # Compile the model with optimizer, loss function, and metrics
    model.compile(optimizer = opt, loss='mae', metrics=['mae'])

    # Print model summary
    model.summary()

    return model


input_shape = np.shape(training_data_X[1]) # Shape of the spectra
d_model = 64  # Dimension of the model
num_heads = 10  # Number of heads of attention
ff_dim = 128  # Dimension of feed-forward layer
num_encoders = 2  # Number of encoders in the Transformer
dropout = 0.1  # Dropout value

CNN = define_model(input_shape, d_model, num_heads, ff_dim, num_encoders, dropout)

Convolutional Neural network

In [ ]:
import tensorflow.keras.optimizers as keras_optimizers

def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(3, 15, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(3, 13, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))
    opt = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=opt, loss='mae', metrics=['mae'])

    return model
input_shape = np.shape(training_data_X[1])
CNN = create_model(input_shape)

Convolutional + LSTM

In [ ]:
import tensorflow.keras.optimizers as keras_optimizers
def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(3, 15, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(3, 13, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(32, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))

    opt = keras_optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mae', metrics=['mae'])

    return model
input_shape = np.shape(training_data_X[1])
CNN = create_model(input_shape)


Convolutional + GRU

In [ ]:
import tensorflow.keras.optimizers as keras_optimizers
def create_model(input_shape):
    model = Sequential()
    model.add(Conv1D(15, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(GRU(64, activation='relu', return_sequences=True))
    model.add(GRU(32, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))

    opt = keras_optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mae', metrics=['mae'])

    return model
input_shape = np.shape(training_data_X[1])
CNN = create_model(input_shape)

Fully connetted

In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))
    opt = keras_optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=opt, loss='mae', metrics=['mae'])
    model.summary()

    return model
input_shape = np.shape(training_data_X[1])
CNN = create_model(input_shape)

In [ ]:
n_epochs = 100
batch_size = 8
#fit the model
history = CNN.fit(train_FH, train_y_FH,
                  batch_size=batch_size,
                  validation_split=0.3,
                  epochs=n_epochs, verbose=1)


In [ ]:
#plot loss and val loss
n=0
epochs = range(n, len(history.history['loss']))
plt.figure(figsize=(10, 6))
plt.title('Cross Entropy Loss')
plt.plot(epochs, history.history['loss'][n:], color='blue', label='train')
plt.plot(epochs, history.history['val_loss'][n:], color='orange', label='validation')
plt.xlabel('Epoch')
plt.legend()

plt.show()
plt.clf()

In [ ]:
#compute outputs
y_pred= CNN.predict(test_FH, verbose = True, batch_size=64)

3/3 [==============================] - 1s 116ms/step


In [ ]:
#plot prediction
x = np.linspace(0, 1, 10000)
plt.figure(figsize=(16, 16))
plt.grid()
plt.plot(x, x, label='Linea di riferimento')
plt.plot(test_y_FH, y_pred, 'x', label='Valori predetti')

plt.xlabel('Rapporto Fe/H reale')
plt.ylabel('Rapporto Fe/H predetto')
plt.title('Confronto tra valori predetti e reali per il rapporto Fe/H')

plt.legend()
plt.show()

In [ ]:
#denormalization of prediction and labels
y_pred = scaler_FH.inverse_transform(y_pred)
test_y_FH_naturali = scaler_FH.inverse_transform(test_y_FH)

In [ ]:
#compute error and variance of prediction and labels
errore , varianza = calcola_errore(y_pred, test_y_FH_naturali)
errore, varianza

(0.035783169059795436, 0.16632602795450982)

In [ ]:
#plot prediction without outliers
filter_and_plot(y_pred, test_y_FH_naturali, -2, 2)

In [ ]:
#Compute MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_y_FH_naturali, y_pred)
print("Mean Absolute Error (MAE):", mae)

##prediction only FeH Rosse

In [ ]:
#compute outputs only for red stars and denormalization of that
y_pred_rosse = CNN.predict(train_rosse_FH, verbose = True, batch_size=64)
y_pred_rosse = scaler_FH.inverse_transform(y_pred_rosse)

In [ ]:
#compute error and variance
errore , varianza = calcola_errore(y_pred_rosse, y_train_rosse_FH)
errore, varianza

In [ ]:
#plot prediction
x = np.linspace(0, 1, 10000)
plt.figure(figsize=(16, 16))
plt.grid()
plt.plot(x, x, label='Linea di riferimento')
plt.plot(y_test_rosse_FH, y_pred, 'x', label='Valori predetti')

plt.xlabel('Rapporto Fe/H reale')
plt.ylabel('Rapporto Fe/H predetto')
plt.title('Confronto tra valori predetti e reali per il rapporto Fe/H')
plt.legend()
plt.show()

In [ ]:
#plot prediction without outliers
filter_and_plot(y_pred, y_test_rosse_FH, 2, segno= False)

## predict onlyFe/H of not red star

In [ ]:
#compute outputs only for red stars and denormalization of that
y_pred_azzurre = CNN.predict(train_azzurre_FH, verbose = True, batch_size=64)
y_pred_azzurre = scaler_FH.inverse_transform(y_pred_azzurre)

7/7 [==============================] - 1s 176ms/step


In [ ]:
#compute error and variance
errore , varianza = calcola_errore(y_pred_azzurre, y_train_azzurre_FH)
errore, varianza

##comparing the performance

In [ ]:
#plot three error gaussian
plotta_gaussiana_errore(y_pred,  y_pred_azzurre, y_pred_rosse, test_y_FH_naturali, y_train_azzurre_FH, y_train_rosse_FH)